In [12]:
from datasets import load_dataset

# Specify the dataset name
dataset_name = "ktgiahieu/maccrobat2018_2020"  # Replace with the desired dataset name

# Download the dataset
dataset = load_dataset(dataset_name)

# Access the dataset
MACCROBAT_train_dataset = dataset["train"]
# test_dataset = dataset["test"]

In [13]:
print()

['A', '68', '-', 'year', '-', 'old', 'female', 'nonsmoker', ',', 'nondrinker', 'with', 'a', 'medical', 'history', 'of', 'hypertension', 'presented', 'with', 'new', '-', 'onset', 'painless', 'jaundice', 'and', 'pruritus', ',', 'a', 'three', '-', 'month', 'history', 'of', '9.9', 'kg', 'weight', 'loss', 'and', 'chronic', 'diarrhea', 'with', 'four', 'to', 'five', 'loose', 'bowel', 'movements', 'per', 'day', '.', '\n', 'Medications', 'included', 'vitamin', 'D', ',', 'amlodipine', 'and', 'eprosartan', '.', '\n', 'Physical', 'examination', 'was', 'normal', 'except', 'for', 'jaundice', 'and', 'muscle', 'wasting', '.', '\n', 'Recent', 'colonoscopy', 'had', 'been', 'normal', '.', '\n', 'Total', 'and', 'direct', 'bilirubin', 'levels', 'were', '6.84', 'mg', '/', 'dL', '(', '116.96', 'μmol', '/', 'L', ')', 'and', '9.18', 'mg', '/', 'dL', '(', '156.98', 'μmol', '/', 'L', ')', ',', 'respectively', '.', '\n', 'Other', 'results', 'included', 'an', 'international', 'normalized', 'ratio', 'of', '1.0', ',

In [16]:
import os

# Your original dataset
tokens = MACCROBAT_train_dataset['tokens'][0]
ner_tags = MACCROBAT_train_dataset['tags'][0]

# Step 1: Identify unique entity classes
unique_entity_classes = set(tag.split('-')[-1] for tag in ner_tags if '-' in tag)

# Step 2: Create separate datasets for each entity class
for entity_class in unique_entity_classes:
    entity_dataset_tokens = []
    entity_dataset_ner_tags = []

    for token, tag in zip(tokens, ner_tags):
        if '-' in tag:
            current_entity = tag.split('-')[-1]
            if current_entity == entity_class:
                # Retain only 'B', 'I', and 'O' tags
                ner_tag = 'B' if tag.startswith('B') else 'I' if tag.startswith('I') else 'O'
                entity_dataset_tokens.append(token)
                entity_dataset_ner_tags.append(ner_tag)
        else:
            # If it's an 'O' tag, retain it
            entity_dataset_tokens.append(token)
            entity_dataset_ner_tags.append('O')

    # Step 3: Save the new dataset to a separate file
    output_folder = 'output_datasets'
    os.makedirs(output_folder, exist_ok=True)
    output_filename = os.path.join(output_folder, f'{entity_class}_dataset.txt')

    with open(output_filename, 'w', encoding='utf-8') as f:
        for token, ner_tag in zip(entity_dataset_tokens, entity_dataset_ner_tags):
            f.write(f'{token}\t{ner_tag}\n')

print("Separate datasets created and saved in the 'output_datasets' folder.")


Separate datasets created and saved in the 'output_datasets' folder.


In [19]:
import os
from collections import defaultdict

# Your dataset with multiple texts
all_texts = MACCROBAT_train_dataset

# Step 1: Identify unique entity classes
unique_entity_classes = set()
for text in all_texts:
    ner_tags = text['tags']
    unique_entity_classes.update(tag.split('-')[-1] for tag in ner_tags if '-' in tag)

# Step 2 and 3: Create separate datasets and save them for each entity class
for entity_class in unique_entity_classes:
    entity_dataset_tokens = defaultdict(list)
    entity_dataset_ner_tags = defaultdict(list)

    for idx, text in enumerate(all_texts):
        tokens = text['tokens']
        ner_tags = text['tags']

        for token, tag in zip(tokens, ner_tags):
            if '-' in tag:
                current_entity = tag.split('-')[-1]
                if current_entity == entity_class:
                    # Retain only 'B', 'I', and 'O' tags
                    ner_tag = 'B' if tag.startswith('B') else 'I' if tag.startswith('I') else 'O'
                    entity_dataset_tokens[idx].append(token)
                    entity_dataset_ner_tags[idx].append(ner_tag)
            else:
                # If it's an 'O' tag, retain it
                entity_dataset_tokens[idx].append(token)
                entity_dataset_ner_tags[idx].append('O')

    # Save the new dataset to a single file for each entity class
    output_folder = 'output_datasets'
    os.makedirs(output_folder, exist_ok=True)
    output_filename = os.path.join(output_folder, f'{entity_class}_dataset.txt')

    with open(output_filename, 'w', encoding='utf-8') as f:
        for idx in range(len(all_texts)):
            for token, ner_tag in zip(entity_dataset_tokens[idx], entity_dataset_ner_tags[idx]):
                f.write(f'{token}\t{ner_tag}\n')
            # Add a newline between texts
            f.write('\n')

print("Separate datasets created and saved in the 'output_datasets' folder.")


Separate datasets created and saved in the 'output_datasets' folder.


In [35]:
import os
from sklearn.model_selection import train_test_split
from collections import defaultdict

# Your dataset with multiple texts
all_texts = MACCROBAT_train_dataset

# Step 1: Identify unique entity classes
unique_entity_classes = set()
for text in all_texts:
    ner_tags = text['tags']
    unique_entity_classes.update(tag.split('-')[-1] for tag in ner_tags if '-' in tag)

# Step 2 and 3: Create separate datasets and save them for each entity class
for entity_class in unique_entity_classes:
    entity_dataset_tokens = defaultdict(list)
    entity_dataset_ner_tags = defaultdict(list)

    for idx, text in enumerate(all_texts):
        tokens = text['tokens']
        ner_tags = text['tags']

        for token, tag in zip(tokens, ner_tags):
            if '-' in tag:
                current_entity = tag.split('-')[-1]
                if current_entity == entity_class:
                    # Retain only 'B', 'I', and 'O' tags
                    ner_tag = 'B' if tag.startswith('B') else 'I' if tag.startswith('I') else 'O'
                    entity_dataset_tokens[idx].append(token)
                    entity_dataset_ner_tags[idx].append(ner_tag)
            else:
                # If it's an 'O' tag, retain it
                entity_dataset_tokens[idx].append(token)
                entity_dataset_ner_tags[idx].append('O')

    # Combine tokens and NER tags for each text
    entity_datasets = []
    for idx in range(len(all_texts)):
        combined_data = list(zip(entity_dataset_tokens[idx], entity_dataset_ner_tags[idx]))
        entity_datasets.append(combined_data)

    # Split the dataset into train, test, and development sets
    train_data, test_dev_data = train_test_split(entity_datasets, test_size=0.15, random_state=42)
    test_data, devel_data = train_test_split(test_dev_data, test_size=0.5, random_state=42)

    # Save the datasets to separate files
    output_folder = 'output_datasets'
    os.makedirs(output_folder, exist_ok=True)

    for dataset, split_name in zip([train_data, test_data, devel_data], ['train', 'test', 'devel']):
        subfolder_path = os.path.join(output_folder, f'{entity_class}')
        os.makedirs(subfolder_path, exist_ok=True)
        output_filename = os.path.join(subfolder_path, f'{split_name}.txt')

        with open(output_filename, 'w', encoding='utf-8') as f:
            for combined_data in dataset:
                for token, ner_tag in combined_data:
                    f.write(f'{token} {ner_tag}\n')  # Single space as the separator
                # Add a newline between texts
                f.write('\n')

print("Train, test, and devel datasets created and saved in the 'output_datasets' folder.")


Train, test, and devel datasets created and saved in the 'output_datasets' folder.


In [ ]:
from datasets import load_dataset

# Specify the dataset name
dataset_name = "tner/bc5cdr"  # Replace with the desired dataset name

# Download the dataset
dataset = load_dataset(dataset_name)

# Access the dataset
bc5cdr_train_dataset = dataset["train"]

In [ ]:
# Specify the dataset name
dataset_name = "jnlpba"  # Replace with the desired dataset name

# Download the dataset
dataset = load_dataset(dataset_name)

# Access the dataset
jnlpba_train_dataset = dataset["train"]

In [ ]:
# Specify the dataset name
dataset_name = "bc2gm_corpus"  # Replace with the desired dataset name

# Download the dataset
dataset = load_dataset(dataset_name)

# Access the dataset
bc2gm_corpus_train_dataset = dataset["train"]